In [1]:
import os
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain.memory.buffer import ConversationBufferMemory
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [13]:
EXAMPLES = [
    {
        "query": "Product build for consumers which involes of processing of personal data",
        "answer": "Guidelines: Data security and Privacy rights.",
        "explanation": """
        Since personal data being collected, people have the right to know how we are collecting their personal data and how long we are gonna store it. 
        We also need to make sure proper measures are in place to protect their data and we have proper encrpytion in place
        """
    }
]

In [14]:
prompt = PromptTemplate(
    template="""
    Role: An Expert General Data Protection Regulation (GDPR) expert which tells which guideline(s) to follow when building an application or product.
    Instructions: Answer user query which guide line to follow. Take help from provided examples and don't make up any answer from your side. First completely analyse the question and properly think befor giving answer and only provide guideline name(s).
    Question: {query}
    Answer: {answer}
    Explanation: {explanation}
    """,
    input_variables=["query", "answer", "explanation"]
)

In [15]:
print(prompt.format(**EXAMPLES[0]))


    Role: An Expert General Data Protection Regulation (GDPR) expert which tells which guideline(s) to follow when building an application or product.
    Instructions: Answer user query which guide line to follow. Take help from provided examples and don't make up any answer from your side. First completely analyse the question and properly think befor giving answer and only provide guideline name(s).
    Question: Product build for consumers which involes of processing of personal data
    Answer: Guidelines: Data security and Privacy rights.
    Explanation: 
        Since personal data being collected, people have the right to know how we are collecting their personal data and how long we are gonna store it. 
        We also need to make sure proper measures are in place to protect their data and we have proper encrpytion in place
        
    


In [16]:
few_shot_template = FewShotPromptTemplate(
    example_prompt=prompt,
    examples=EXAMPLES,
    suffix="Question: {query}",
    input_variables=["query"]
)

In [17]:
print(few_shot_template.format(query="Ask any question"))


    Role: An Expert General Data Protection Regulation (GDPR) expert which tells which guideline(s) to follow when building an application or product.
    Instructions: Answer user query which guide line to follow. Take help from provided examples and don't make up any answer from your side. First completely analyse the question and properly think befor giving answer and only provide guideline name(s).
    Question: Product build for consumers which involes of processing of personal data
    Answer: Guidelines: Data security and Privacy rights.
    Explanation: 
        Since personal data being collected, people have the right to know how we are collecting their personal data and how long we are gonna store it. 
        We also need to make sure proper measures are in place to protect their data and we have proper encrpytion in place
        
    

Question: Ask any question


In [18]:
openai_llm = OpenAI(temperature=0.4)

In [19]:
chain = (
    {"query": RunnablePassthrough()}
    | few_shot_template
    | openai_llm
)

In [20]:
ans = chain.invoke(
    """
    We are building an online book store through which person can order book. For that we collection information like person's name, address, phone number.
    Let me know which GDPR guidelines I need to follow?
    """
)

print(ans)


Answer: Guidelines: Lawfulness, fairness and transparency, Purpose limitation, Data minimization, Accuracy, Storage limitation, Integrity and confidentiality, and Accountability.
Explanation: 
    Since personal data is being collected, people have the right to know how we are collecting their personal data and for what purpose. We should only collect the necessary information and make sure it is accurate. We also need to have proper measures in place to protect the data and only store it for as long as necessary. Finally, we need to be accountable for how we handle and process personal data.
